In [2]:
from cffi import FFI

In [3]:
ffi = FFI()
ffi.cdef("""
    long __stdcall IV_open();
    long __stdcall IV_close();
    long __stdcall IV_selectdevice(long *devnr);
    long __stdcall IV_connect(long* devconnect);
    long __stdcall IV_readSN(char *sntext);
    long __stdcall IV_getdevicestatus(long *devstatus);
    long __stdcall IV_getcellstatus(long *devcellstatus);
    long __stdcall IV_setcellon(long *cellon);
    long __stdcall IV_getcurrent(double *value);
    long __stdcall IV_setcurrent(double *value);
    long __stdcall IV_setpotential(double *value);
    long __stdcall IV_getpotential(double *value);
    long __stdcall IV_readmethod(char* fname);
    long __stdcall IV_startmethod(char *fname);
    long __stdcall IV_abort();
    
""")


In [4]:
connflag = ffi.new("long *", 1)
pchar16 = ffi.new("char[]", 16)
lib = ffi.dlopen("../pyvium_core/IVIUM_remdriver64.dll")
lib.IV_open()
rc = lib.IV_connect(connflag)
print(rc)
if rc == 0:
    rc = lib.IV_readSN(pchar16)
    hw_version = ffi.string(pchar16).decode("utf-8") 
    print('Serial num:',hw_version)
if rc == -1 or not hw_version:
    print("no IviumSoft.")
if rc == 1 or not hw_version:
    print("no device.")


-1
no IviumSoft.


NameError: name 'hw_version' is not defined

Get potential (OCP)

In [ ]:
def IV_getpotential():
    pdouble = ffi.new("double *")
    rc = lib.IV_getpotential(pdouble)
    return pdouble[0]
IV_getpotential()



0.000339746504323557